## Notebook to analyze Saliency map and Physical propoerties
TGRS [Kurihana et al. 2021](https://ieeexplore.ieee.org/document/9497325)  Appendix.F  
Copy from `118-SailencyMap-TF2.ipynb`

In [2]:
%matplotlib inline
import os
import cv2
import sys
import pickle
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.python.keras.models import Model

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from pyhdf.SD import SD, SDC

print('tensorflow {}'.format(tf.__version__))
print("keras {}".format(keras.__version__))

tensorflow 1.14.0
keras 2.2.4-tf


In [3]:
libdir = '/home/tkurihana/Research/clouds/src_analysis/lib_hdfs'
sys.path.insert(1,os.path.join(sys.path[0],libdir))
from analysis_lib import mod06_proc_sds

In [66]:
import re
import gc
import glob
import pandas as pd

In [46]:
from scipy.stats import pearsonr
from scipy.special import kl_div

### Load grads dictionary

In [4]:
with open("./saliency-grads_dict.pkl", 'rb') as f:
    grads_dict = pickle.load(f)

#### resize 32x32 to 128x128

In [6]:
grads_patch_dict = {}
for key, spatches in grads_dict.items():
    rspatches = tf.image.resize(spatches, (128,128))
    img = tf.keras.backend.eval(rspatches)
    grads_patch_dict[key] = img

In [7]:
for key, spatches in grads_patch_dict.items():
    print(spatches.shape)

(240, 128, 128, 6)
(240, 128, 128, 6)
(240, 128, 128, 6)
(240, 128, 128, 6)
(240, 128, 128, 6)
(240, 128, 128, 6)


### Extract MOD06 files & patches

In [8]:
sfiles = np.load('top20-selectedfiles.npy')

In [10]:
scoords = np.load('./top20-selectedcoords.npy')

In [32]:
def get_filepath(filepath, datadir, prefix='',ext='.hdf'):
    bname = os.path.basename(filepath) # ex. 2017213.2355
    dateinfo = re.findall('[0-9]{7}.[0-9]{4}' , bname)
    date     = dateinfo[0].rstrip("['").lstrip("']")
    filelist = glob.glob(datadir+'/'+prefix+date+f'*{ext}')
    try:
        if len(filelist) > 0:
            return filelist[0]
    except:
        efile = datadir+'/'+prefix+date+'*.hdf'
        print(" Program will be forcibly terminated", flush=True)
        raise NameError(" ### File Not Found: No such file or directory "+str(efile)+" ### ")

In [15]:
myd06_datadir="/home/tkurihana/Research/data/MYD06/20080101"

In [18]:
prefix='MYD06_L2.A'

In [37]:
param_keys=["Cloud_Optical_Thickness",
            "Cloud_Phase_Infrared_1km",
            "cloud_top_pressure_1km",
            "Cloud_Effective_Radius"]

In [38]:
patch_size=128

In [41]:
COTs_list = []
CPHs_list = []
CTPs_list = []
CERs_list = []
for idx, (ifile, (x,y)) in enumerate(zip(sfiles, scoords)):
    try:
        myd06 = get_filepath(ifile, myd06_datadir, prefix=prefix,ext='.hdf')
    except:
        print('NOT FOUND')
        myd06 = None
        pass
    
    if myd06:
        myd06_hdf = SD(myd06, SDC.READ)
        tmp_dict = {}
        for param in param_keys:
            sds_array = myd06_hdf.select(param)
            tmp_array = mod06_proc_sds(sds_array, variable=param)
            sarray = tmp_array[x:x+patch_size, y:y+patch_size]
            sarray = np.expand_dims(sarray,axis=0)
            
            if param == param_keys[0]:
                COTs_list.append(sarray)
            elif param == param_keys[1]:
                CPHs_list.append(sarray)
            elif param == param_keys[2]:
                CTPs_list.append(sarray)
            elif param == param_keys[3]:
                CERs_list.append(sarray)
                

/home/tkurihana/Research/clouds/src_analysis/lib_hdfs/analysis_lib.py:361: RuntimeWarning: invalid value encountered in greater
  err_idx = np.where(array > 100.0) # optical thickness range[0,100] no unit


In [42]:
len(COTs_list), len(CPHs_list), len(CTPs_list), len(CERs_list)

(240, 240, 240, 240)

List to Patches

In [43]:
cot_array = np.concatenate(COTs_list, axis=0)
cph_array = np.concatenate(CPHs_list, axis=0)
ctp_array = np.concatenate(CTPs_list, axis=0)
cer_array = np.concatenate(CERs_list, axis=0)

In [44]:
cot_array.shape

(240, 128, 128)

Save data

In [45]:
np.save('./top20-selected-COT', cot_array)
np.save('./top20-selected-CPH', cph_array)
np.save('./top20-selected-CTP', ctp_array)
np.save('./top20-selected-CER', cer_array)

### Analysis Corr & KLD

In [59]:
def calc_metrics(grad, phys,):
    """ grad (#, 128,128) after select one band
        phys (#, 128,128)
    """
    valid_idx = np.where(~np.isnan(phys))
    _grad = grad[valid_idx].ravel()
    _phys = phys[valid_idx].ravel()
    corr, p_val = pearsonr(_grad, _phys)
    
    vmin = np.min(_grad)
    _grad+=vmin # avoid inf
    kld = kl_div(_grad, _phys)
    return {'corr':corr, 'p_val':p_val, 'kld':kld}

In [50]:
def _run_metrics(grads, phys_dict,
                 bands = [5,7,20,28,29,31],
                 param_keys = None,
                ):
    results = dict()
    for param in param_keys:
        phys = phys_dict[param]
        for idx, band in enumerate(bands):
            grad = grads[:,:,:,idx]
            results[f'{param}-{band}'] = calc_metrics(grad, phys,)
    return results

In [51]:
def run_associate_test(grads_dict, phys_dict,
                       param_keys = None,
                      ):
    results_dict = dict()
    for layer, grads in grads_dict.items():
        print(f"ANALYSIS {layer} START")
        results_dict[layer] = _run_metrics(grads, phys_dict,
                                            bands = [5,7,20,28,29,31],
                                            param_keys = param_keys)
    return results_dict

In [52]:
phys_dict = {
            "Cloud_Optical_Thickness":cot_array,
            "Cloud_Phase_Infrared_1km":cph_array,
            "cloud_top_pressure_1km":ctp_array,
            "Cloud_Effective_Radius":cer_array
}

In [60]:
results_dict = run_associate_test(grads_patch_dict, phys_dict,param_keys = param_keys)

ANALYSIS leaky_re_lu START
ANALYSIS leaky_re_lu_3 START
ANALYSIS leaky_re_lu_6 START
ANALYSIS leaky_re_lu_9 START
ANALYSIS leaky_re_lu_12 START
ANALYSIS leaky_re_lu_14 START


In [64]:
results_dict.keys()

dict_keys(['leaky_re_lu', 'leaky_re_lu_3', 'leaky_re_lu_6', 'leaky_re_lu_9', 'leaky_re_lu_12', 'leaky_re_lu_14'])

In [65]:
results_dict['leaky_re_lu'].keys()

dict_keys(['Cloud_Optical_Thickness-5', 'Cloud_Optical_Thickness-7', 'Cloud_Optical_Thickness-20', 'Cloud_Optical_Thickness-28', 'Cloud_Optical_Thickness-29', 'Cloud_Optical_Thickness-31', 'Cloud_Phase_Infrared_1km-5', 'Cloud_Phase_Infrared_1km-7', 'Cloud_Phase_Infrared_1km-20', 'Cloud_Phase_Infrared_1km-28', 'Cloud_Phase_Infrared_1km-29', 'Cloud_Phase_Infrared_1km-31', 'cloud_top_pressure_1km-5', 'cloud_top_pressure_1km-7', 'cloud_top_pressure_1km-20', 'cloud_top_pressure_1km-28', 'cloud_top_pressure_1km-29', 'cloud_top_pressure_1km-31', 'Cloud_Effective_Radius-5', 'Cloud_Effective_Radius-7', 'Cloud_Effective_Radius-20', 'Cloud_Effective_Radius-28', 'Cloud_Effective_Radius-29', 'Cloud_Effective_Radius-31'])

In [74]:
df = pd.DataFrame()
for key, val in results_dict.items():
    clist = [] 
    for _key in results_dict['leaky_re_lu'].keys():
        clist.append(np.abs(val[_key]['corr'])) # abs corr
    carray = np.asarray(clist)
    df[key] = carray
df.index = results_dict['leaky_re_lu'].keys()

In [79]:
df.style.highlight_max(color = 'lightgreen', axis = 1) #.apply(
    #lambda x: ["background: lightgreen" if v > x.iloc[0] else "" for v in x], axis = 1)

,leaky_re_lu,leaky_re_lu_3,leaky_re_lu_6,leaky_re_lu_9,leaky_re_lu_12,leaky_re_lu_14
Cloud_Optical_Thickness-5,0.027839,0.016942,0.002151,0.007348,0.000254,0.000538
Cloud_Optical_Thickness-7,0.067562,0.018258,0.024242,0.016791,0.010124,0.014734
Cloud_Optical_Thickness-20,0.113732,0.000038,0.017734,0.012883,0.031266,0.003795
Cloud_Optical_Thickness-28,0.002953,0.025758,0.006364,0.006212,0.001800,0.005238
Cloud_Optical_Thickness-29,0.032907,0.009567,0.007163,0.009072,0.013815,0.004024
Cloud_Optical_Thickness-31,0.091804,0.039295,0.019632,0.012732,0.003171,0.006075
Cloud_Phase_Infrared_1km-5,0.178691,0.052498,0.082845,0.070017,0.045022,0.042347
Cloud_Phase_Infrared_1km-7,0.127952,0.086444,0.027135,0.007062,0.004083,0.004723
Cloud_Phase_Infrared_1km-20,0.372043,0.315710,0.278829,0.224769,0.200161,0.129095
Cloud_Phase_Infrared_1km-28,0.036048,0.042208,0.084904,0.109335,0.089480,0.040395
